<a href="https://colab.research.google.com/github/Surendra58/Suri/blob/master/NEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NEAT on Gym Environments

## Setup

In [1]:
!pip install neat-python graphviz box2d

     |████████████████████████████████| 51kB 3.1MB/s 
     |████████████████████████████████| 1.3MB 11.2MB/s 


In [2]:
!git clone https://github.com/theneuralbeing/neat

Cloning into 'neat'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 35 (delta 14), reused 30 (delta 9), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [3]:
cd /content/neat

/content/neat


## Train

In [4]:
%%time
# Run the training script
!python train.py


 ****** Running generation 0 ****** 


Elapsed Time: 14.2140052318573

Population's average fitness: -468.52260 stdev: 224.58026
Best fitness: -118.75508 - size: (4, 32) - species 1 - id 45
Average adjusted fitness: 0.635
Mean genetic distance 1.155, standard deviation 0.182
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50   -118.8    0.635     0
Total extinctions: 0
Generation time: 14.196 sec

 ****** Running generation 1 ****** 


Elapsed Time: 26.29949951171875

Population's average fitness: -310.10670 stdev: 171.56323
Best fitness: -104.32807 - size: (5, 33) - species 1 - id 65
Average adjusted fitness: 0.669
Mean genetic distance 1.241, standard deviation 0.218
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    50   -104.3    0.669     0
Total extinctions: 0
Generation time: 12.086 sec (13.141 average)


## Test

### Setup Colab to Render the Gym Environment

In [5]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [6]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [7]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [8]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

### Run the Agent

In [9]:
import neat
import pickle

In [10]:
def play_agent(genome, config, episodes=5, max_episode_length=None, render=False):
    env = wrap_env(gym.make("LunarLander-v2"))
    
    agent = neat.nn.FeedForwardNetwork.create(genome, config)
    
    total_rewards = []
    
    for ep in range(episodes):
        observation = env.reset()
        if max_episode_length is not None:
            env._max_episode_steps = max_episode_length
        
        episodic_reward = 0
        
        while(1):
            if render:
                env.render(mode='rgb_array')
            
            action = np.array(agent.activate(observation)).argmax()
            observation, reward, done, info = env.step(action)
            
            episodic_reward += reward
            if done:
                break
        
        
        total_rewards.append(episodic_reward)
    
    env.close()
    print('Mean Rewards across all episodes', np.array(total_rewards).mean())
    print('Best Reward in any single episode', max(total_rewards))

In [11]:
# Load the configuration file
config_file = 'config-feedforward.txt'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_file)

In [12]:
# Load the Best Genome
genome = pickle.load(open("best.genome", "rb"))

In [13]:
play_agent(genome, config, episodes=3, render=True)

Mean Rewards across all episodes 89.25249958747138
Best Reward in any single episode 238.18617325360984


In [14]:
show_video()